In [1]:
# Import required libraries
from datetime import datetime
import csv
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# Generating a URL with a function
def generate_url(position, location):
    base_url = "https://example.com/jobs?q={}&l={}"  # Example URL template
    # Replace example URL with the actual URL template of the job posting site
    url = base_url.format(position, location)
    return url

In [3]:
# Extract the Job Data from a single job posting card
def extract_job_data(job_posting):
    try:
        job_title = job_posting.find('h2', class_='job-title').text.strip()
    except AttributeError:
        job_title = ""
    
    try:
        company_name = job_posting.find('span', class_='company-name').text.strip()
    except AttributeError:
        company_name = ""
    
    try:
        location = job_posting.find('span', class_='location').text.strip()
    except AttributeError:
        location = ""
    
    # Continue extracting other relevant job data
    
    return {
        'Job Title': job_title,
        'Company': company_name,
        'Location': location
        # Add more fields as needed
    }

In [4]:
# Define the main function
def main(job_position, location):
    # 1. Set headers
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    # 2. Construct URL
    url = generate_url(job_position, location)
    
    # 3. Send HTTP request and retrieve HTML
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 4. Parse HTML and select job postings
    job_postings = soup.find_all('div', class_='job-posting')
    
    job_data_list = []
    
    # 5. Extract job posting information
    for job_posting in job_postings:
        job_data = extract_job_data(job_posting)
        job_data_list.append(job_data)
    
    # 6. Write data to CSV file
    filename = f"{job_position}_{location}_jobs_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Job Title', 'Company', 'Location']  # Add more fields as needed
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for job_data in job_data_list:
            writer.writerow(job_data)
    
    # 7. Print success message
    print("Job data extraction successful. CSV file created:", filename)
